In [191]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [192]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile1)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=middle_percentile2)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [193]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [194]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [195]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [196]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [197]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator])

## Segment Annual Total Savings

In [198]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [199]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [200]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [201]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']])

In [202]:
community = 'jackson_county'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [203]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Heat Pump Water Heater                                              3269.1
Basic Enclosure                                                     4855.9
Enhanced Enclosure                                                  6941.2
Mininum Efficiency Heat Pump with Existing Heat Backup             16880.9
Mininum Efficiency Heat Pump with Electric Heat Backup             17362.6
Mininum Efficiency Whole Home Electrification                      17468.6
High Efficiency Heat Pump with Electric Heat Backup                28856.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        33594.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     34484.3
High Efficie

## 1.2 Total Annual Savings

In [204]:
figure_total_savings_1(community_results)

In [205]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [206]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [207]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Mininum Efficiency Whole Home Electrification                      17646.462117
Electric Clothes Dryer                                              3591.614938
Electric Cooking                                                    9265.180107
Induction Cooking                                                  27538.893286
Heat Pump Clothes Dryer                                            23302.660882
Mininum Efficiency Heat Pump with Electric Heat Backup             27468.480932
Heat Pump Water Heater                                             27540.061448
Mininum Efficiency Heat Pump with Existing Heat Backup             26996.819723
Basic Enclosure                                                    27181.016959
Enhanced Enclosure                                                 27284.784354
High Efficiency Heat Pump with Electric Heat Backup                27468.480932
High Efficiency Whole Home Electrification                         27723.000000
Basic Enclosure + HPWH + Hi

In [208]:
metric = "saving_cbill.total_usd"
print("Community summary:")
results = get_summary(community_results, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
saving_results['whole_community_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
saving_results['whole_community_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
saving_results['whole_community_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
saving_results['whole_community_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
results

Community summary:


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1147,27723.000000,969.116308,178.248036,380.642562,1347.068175,2119.018933
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1147,27723.000000,951.961947,174.466762,377.292809,1323.091279,2077.911943
Basic Enclosure + High Efficiency Whole Home Electrification,1147,27723.000000,946.652588,175.097136,366.410318,1323.635767,2108.064584
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1147,27723.000000,929.498227,174.032757,364.978053,1290.691076,2053.322462
High Efficiency Whole Home Electrification,1147,27723.000000,789.075241,97.953658,302.802793,1110.354666,1790.090722
High Efficiency Heat Pump with Electric Heat Backup,1144,27468.480932,643.469980,57.878462,220.506818,935.637497,1555.435624
Enhanced Enclosure,1137,27284.784354,405.739468,47.501294,111.923211,573.286352,922.384096
Basic Enclosure,1131,27181.016959,331.293323,39.468827,94.142048,479.189436,771.355599
Mininum Efficiency Heat Pump with Existing Heat Backup,1135,26996.819723,249.797553,-193.323643,-8.896191,311.283058,895.007211


# 2. Results for Segments

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, renters, all income 

Segment 2. Occupied, Multifamily buildings, pre 1940, renters, all income 

Segment 3. Occupied, All housing types, renters, all income  


In [209]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

In [210]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']

seg2_building_type = ['Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units']
seg2_vintage = ['<1940']
seg2_tenure = ['Renter']

seg3_tenure = ['Renter']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure),
                     'segment'] = 'segment2'

seg3 = community_results.copy().assign(segment='segment3')
seg3 = seg3.loc[seg3['build_existing_model.tenure'].isin(seg3_tenure) &
                seg3['build_existing_model.vacancy_status'].isin(vacancy_status)]
community_results = pd.concat([community_results, seg3], axis=0)

In [211]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: segment1: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,25,621.890644,1019.549741,-134.432152,745.907553,1603.227955,1677.645841
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,25,621.890644,1007.374649,-156.409969,734.653035,1648.157812,1864.195102
Basic Enclosure + High Efficiency Whole Home Electrification,25,621.890644,987.918776,-265.237315,745.907553,1603.227955,1677.645841
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,25,621.890644,975.743684,-287.215132,734.653035,1648.157812,1864.195102
High Efficiency Whole Home Electrification,25,621.890644,740.076283,-601.435602,461.711484,1478.048648,1700.874412
High Efficiency Heat Pump with Electric Heat Backup,25,621.890644,556.468543,-518.757282,352.383981,1030.226154,1449.364612
Enhanced Enclosure,25,621.890644,490.342674,107.727698,301.452366,616.480273,777.411943
Basic Enclosure,25,621.890644,441.250756,100.314590,272.424620,602.950289,663.273025
Mininum Efficiency Heat Pump with Existing Heat Backup,24,543.577085,298.546356,-156.317204,-22.523725,574.984802,1078.569271


Summary for segment: segment2: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,5,202.310028,1070.242967,540.225657,578.037942,1527.758846,2047.329660
Basic Enclosure + High Efficiency Whole Home Electrification,5,202.310028,1055.114809,540.225657,578.037942,1469.137230,2047.329660
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,5,202.310028,1051.374838,537.459713,565.015838,1489.092789,2012.052935
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,5,202.310028,1036.246679,537.459713,565.015838,1430.471174,2012.052935
High Efficiency Whole Home Electrification,5,202.310028,902.307017,343.112961,402.856627,1286.718522,1783.860948
High Efficiency Heat Pump with Electric Heat Backup,5,202.310028,777.299932,253.522748,330.474102,1099.700005,1654.132523
Enhanced Enclosure,5,202.310028,424.689454,71.383489,137.655068,716.325015,902.323274
Mininum Efficiency Heat Pump with Existing Heat Backup,5,202.310028,411.642486,-15.131826,63.186725,768.529187,1049.032945
Basic Enclosure,5,202.310028,382.683153,71.383489,138.080708,611.255989,902.323274


Summary for segment: segment3: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,254,7589.336361,763.296433,224.027737,370.281332,922.057913,1647.085552
Basic Enclosure + High Efficiency Whole Home Electrification,254,7589.336361,750.731252,222.548804,369.399130,891.014769,1640.645881
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,254,7589.336361,747.142161,206.821223,358.803739,906.968678,1649.511860
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,254,7589.336361,734.576981,196.463900,357.678121,872.996265,1621.986220
High Efficiency Whole Home Electrification,254,7589.336361,633.819131,168.174087,309.551870,737.785413,1478.832882
High Efficiency Heat Pump with Electric Heat Backup,251,7334.817293,486.501730,119.553017,219.263992,573.053653,1140.626789
Enhanced Enclosure,250,7389.992755,282.053255,43.777069,90.374585,386.005379,623.811893
Basic Enclosure,249,7343.004620,243.556142,37.748651,86.434160,296.702523,574.424688
Mininum Efficiency Heat Pump with Existing Heat Backup,251,7430.929388,191.451963,-33.690228,16.196389,186.472895,752.190796


## 2.1 Upgrade Cost Per Dwelling Unit 

In [212]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Heat Pump Water Heater                                              3157.9
Basic Enclosure                                                     6598.0
Enhanced Enclosure                                                  8984.9
Mininum Efficiency Heat Pump with Existing Heat Backup             22367.8
Mininum Efficiency Whole Home Electrification                      23543.5
Mininum Efficiency Heat Pump with Electric Heat Backup             27504.0
High Efficiency Heat Pump with Electric Heat Backup                32211.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35146.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35996.6
Basic Enclos

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Basic Enclosure                                                     2927.6
Heat Pump Water Heater                                              2964.4
Enhanced Enclosure                                                  3242.4
Mininum Efficiency Whole Home Electrification                      13575.0
Mininum Efficiency Heat Pump with Electric Heat Backup             14153.2
Mininum Efficiency Heat Pump with Existing Heat Backup             14153.2
High Efficiency Heat Pump with Electric Heat Backup                24100.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28268.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     28456.4
High Efficiency Whole Home Electrification                         30223.0
Basic Enclos

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Basic Enclosure                                                     2795.5
Heat Pump Water Heater                                              3035.3
Enhanced Enclosure                                                  4113.6
Mininum Efficiency Heat Pump with Existing Heat Backup             12271.5
Mininum Efficiency Heat Pump with Electric Heat Backup             12971.5
Mininum Efficiency Whole Home Electrification                      13007.9
High Efficiency Heat Pump with Electric Heat Backup                25439.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28374.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     29073.8
High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [213]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]

segment1


upgrade_name
Basic Enclosure                                                    0.06688
Enhanced Enclosure                                                 0.05457
Heat Pump Water Heater                                             0.03917
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02799
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02776
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02513
Basic Enclosure + High Efficiency Whole Home Electrification       0.02487
High Efficiency Whole Home Electrification                         0.01853
High Efficiency Heat Pump with Electric Heat Backup                0.01728
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01335
Heat Pump Clothes Dryer                                            0.00693
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.00387
Induction Cooking                                                 -0.00452
Mininum Effi

segment2


upgrade_name
Enhanced Enclosure                                                 0.13098
Basic Enclosure                                                    0.13071
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03695
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03666
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03385
Basic Enclosure + High Efficiency Whole Home Electrification       0.03357
High Efficiency Heat Pump with Electric Heat Backup                0.03225
High Efficiency Whole Home Electrification                         0.02985
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02908
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02596
Heat Pump Water Heater                                             0.01767
Heat Pump Clothes Dryer                                            0.01159
Induction Cooking                                                  0.00014
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.08713
Enhanced Enclosure                                                 0.06857
Heat Pump Water Heater                                             0.03268
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02589
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02570
Basic Enclosure + High Efficiency Whole Home Electrification       0.02341
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02329
High Efficiency Whole Home Electrification                         0.02025
High Efficiency Heat Pump with Electric Heat Backup                0.01912
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01560
Heat Pump Clothes Dryer                                            0.00907
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00685
Induction Cooking                                                 -0.00101
Mininum Effi

 ### Annual Energy Savings Per Dollar Invested

In [214]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]

segment1


upgrade_name
Basic Enclosure                                                    0.00406
Enhanced Enclosure                                                 0.00345
Heat Pump Water Heater                                             0.00267
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00210
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00208
Basic Enclosure + High Efficiency Whole Home Electrification       0.00192
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00190
Mininum Efficiency Whole Home Electrification                      0.00177
High Efficiency Whole Home Electrification                         0.00173
High Efficiency Heat Pump with Electric Heat Backup                0.00168
Electric Cooking                                                   0.00155
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00135
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00127
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.00525
Enhanced Enclosure                                                 0.00512
Mininum Efficiency Whole Home Electrification                      0.00311
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00152
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00151
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00149
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00139
Basic Enclosure + High Efficiency Whole Home Electrification       0.00139
High Efficiency Heat Pump with Electric Heat Backup                0.00137
Electric Cooking                                                   0.00132
High Efficiency Whole Home Electrification                         0.00129
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00119
Heat Pump Water Heater                                             0.00108
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.00446
Enhanced Enclosure                                                 0.00352
Mininum Efficiency Whole Home Electrification                      0.00255
Heat Pump Water Heater                                             0.00197
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00149
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00147
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00145
Electric Cooking                                                   0.00138
Basic Enclosure + High Efficiency Whole Home Electrification       0.00134
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00132
High Efficiency Whole Home Electrification                         0.00126
High Efficiency Heat Pump with Electric Heat Backup                0.00121
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00095
Electric Clo

 ### Annual Emission Savings Per Dollar Invested

In [215]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]

segment1


upgrade_name
Basic Enclosure                                                    0.29102
Enhanced Enclosure                                                 0.24744
Heat Pump Water Heater                                             0.18518
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14604
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.14482
Basic Enclosure + High Efficiency Whole Home Electrification       0.13359
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.13276
High Efficiency Whole Home Electrification                         0.11783
High Efficiency Heat Pump with Electric Heat Backup                0.11362
Mininum Efficiency Whole Home Electrification                      0.09897
Electric Cooking                                                   0.09888
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09193
Mininum Efficiency Heat Pump with Electric Heat Backup             0.07460
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.40280
Enhanced Enclosure                                                 0.39559
Mininum Efficiency Whole Home Electrification                      0.17830
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.11617
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.11572
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10732
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.10631
Basic Enclosure + High Efficiency Whole Home Electrification       0.10585
High Efficiency Heat Pump with Electric Heat Backup                0.10420
High Efficiency Whole Home Electrification                         0.09744
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09289
Electric Cooking                                                   0.08219
Heat Pump Water Heater                                             0.07273
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.33134
Enhanced Enclosure                                                 0.26257
Mininum Efficiency Whole Home Electrification                      0.14837
Heat Pump Water Heater                                             0.13737
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.10601
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.10456
Basic Enclosure + High Efficiency Whole Home Electrification       0.09653
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.09543
Mininum Efficiency Heat Pump with Electric Heat Backup             0.09433
High Efficiency Whole Home Electrification                         0.08969
Electric Cooking                                                   0.08693
High Efficiency Heat Pump with Electric Heat Backup                0.08619
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06929
Heat Pump Cl

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [216]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    634048.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     626476.9
Basic Enclosure + High Efficiency Whole Home Electrification       614377.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        606805.9
High Efficiency Whole Home Electrification                         460246.5
High Efficiency Heat Pump with Electric Heat Backup                346062.6
Enhanced Enclosure                                                 304939.5
Basic Enclosure                                                    274409.7
Mininum Efficiency Heat Pump with Existing Heat Backup             162283.0
Heat Pump Water Heater                                              76915.8
Heat Pump Clothes Dryer                                             11207.7
Induction Cooking                                                   -5564.1
Electric Cooking                                                    -9250.0

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    216520.9
Basic Enclosure + High Efficiency Whole Home Electrification       213460.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     212703.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        209643.1
High Efficiency Whole Home Electrification                         182545.8
High Efficiency Heat Pump with Electric Heat Backup                157255.6
Enhanced Enclosure                                                  85918.9
Mininum Efficiency Heat Pump with Existing Heat Backup              83279.4
Basic Enclosure                                                     77420.6
Mininum Efficiency Heat Pump with Electric Heat Backup              74342.6
Heat Pump Water Heater                                              10594.5
Heat Pump Clothes Dryer                                              3440.2
Induction Cooking                                                      46.6

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5792913.4
Basic Enclosure + High Efficiency Whole Home Electrification       5697552.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5670313.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5574951.8
High Efficiency Whole Home Electrification                         4810266.6
High Efficiency Heat Pump with Electric Heat Backup                3568401.3
Enhanced Enclosure                                                 2084371.5
Basic Enclosure                                                    1788433.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1422666.0
Heat Pump Water Heater                                              749442.9
Mininum Efficiency Heat Pump with Electric Heat Backup              651775.1
Heat Pump Clothes Dryer                                             119504.7
Induction Cooking                                              

### Annual Total Energy Savings

In [217]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    48042.2
Basic Enclosure + High Efficiency Whole Home Electrification       47473.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     46487.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        45918.5
High Efficiency Whole Home Electrification                         43016.8
High Efficiency Heat Pump with Electric Heat Backup                33721.0
Mininum Efficiency Whole Home Electrification                      23102.2
Mininum Efficiency Heat Pump with Electric Heat Backup             21804.6
Enhanced Enclosure                                                 19297.8
Basic Enclosure                                                    16652.4
Mininum Efficiency Heat Pump with Existing Heat Backup             16422.0
Heat Pump Water Heater                                              5244.3
Heat Pump Clothes Dryer                                              797.6
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    8897.3
Basic Enclosure + High Efficiency Whole Home Electrification       8808.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     8734.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        8645.7
High Efficiency Whole Home Electrification                         7914.9
High Efficiency Heat Pump with Electric Heat Backup                6662.3
Mininum Efficiency Heat Pump with Electric Heat Backup             4265.8
Mininum Efficiency Heat Pump with Existing Heat Backup             3402.7
Enhanced Enclosure                                                 3355.5
Basic Enclosure                                                    3109.8
Mininum Efficiency Whole Home Electrification                      2481.8
Heat Pump Water Heater                                              647.1
Heat Pump Clothes Dryer                                              99.5
Induction Cooking        

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    328653.1
Basic Enclosure + High Efficiency Whole Home Electrification       325895.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     318886.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        316129.4
High Efficiency Whole Home Electrification                         300240.6
High Efficiency Heat Pump with Electric Heat Backup                225789.1
Mininum Efficiency Heat Pump with Electric Heat Backup             141478.3
Mininum Efficiency Whole Home Electrification                      140333.5
Enhanced Enclosure                                                 107095.7
Basic Enclosure                                                     91530.2
Mininum Efficiency Heat Pump with Existing Heat Backup              86498.3
Heat Pump Water Heater                                              45275.6
Heat Pump Clothes Dryer                                              5085.1

### Annual Total Emission Savings

In [218]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3349936.0
Basic Enclosure + High Efficiency Whole Home Electrification       3300204.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3241854.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3192122.4
High Efficiency Whole Home Electrification                         2927159.2
High Efficiency Heat Pump with Electric Heat Backup                2275955.1
Enhanced Enclosure                                                 1382608.0
Mininum Efficiency Whole Home Electrification                      1289453.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1275941.5
Basic Enclosure                                                    1194143.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1117701.3
Heat Pump Water Heater                                              363660.6
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    679971.6
Basic Enclosure + High Efficiency Whole Home Electrification       672969.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     668782.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        661780.3
High Efficiency Whole Home Electrification                         595793.9
High Efficiency Heat Pump with Electric Heat Backup                508076.0
Mininum Efficiency Heat Pump with Electric Heat Backup             307295.4
Mininum Efficiency Heat Pump with Existing Heat Backup             265965.1
Enhanced Enclosure                                                 259493.4
Basic Enclosure                                                    238573.1
Mininum Efficiency Whole Home Electrification                      142159.9
Heat Pump Water Heater                                              43615.0
Heat Pump Clothes Dryer                                              7027.1

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    23735718.7
Basic Enclosure + High Efficiency Whole Home Electrification       23495432.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     23070169.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        22829883.1
High Efficiency Whole Home Electrification                         21310056.9
High Efficiency Heat Pump with Electric Heat Backup                16082601.0
Mininum Efficiency Heat Pump with Electric Heat Backup              8974964.1
Mininum Efficiency Whole Home Electrification                       8178618.5
Enhanced Enclosure                                                  7981917.7
Basic Enclosure                                                     6801407.9
Mininum Efficiency Heat Pump with Existing Heat Backup              6318672.9
Heat Pump Water Heater                                              3150542.1
Heat Pump Clothes Dryer                            

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [219]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1019.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1007.4
Basic Enclosure + High Efficiency Whole Home Electrification        987.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         975.7
High Efficiency Whole Home Electrification                          740.1
High Efficiency Heat Pump with Electric Heat Backup                 556.5
Enhanced Enclosure                                                  490.3
Basic Enclosure                                                     441.3
Mininum Efficiency Heat Pump with Existing Heat Backup              298.5
Heat Pump Water Heater                                              123.7
Heat Pump Clothes Dryer                                              18.0
Induction Cooking                                                    -8.9
Electric Cooking                                                    -25.3
Electric Clothes Dryer   

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1070.2
Basic Enclosure + High Efficiency Whole Home Electrification       1055.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1051.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1036.2
High Efficiency Whole Home Electrification                          902.3
High Efficiency Heat Pump with Electric Heat Backup                 777.3
Enhanced Enclosure                                                  424.7
Mininum Efficiency Heat Pump with Existing Heat Backup              411.6
Basic Enclosure                                                     382.7
Mininum Efficiency Heat Pump with Electric Heat Backup              367.5
Heat Pump Water Heater                                               52.4
Heat Pump Clothes Dryer                                              30.1
Induction Cooking                                                     0.3
Electric Cooking         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    763.3
Basic Enclosure + High Efficiency Whole Home Electrification       750.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     747.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        734.6
High Efficiency Whole Home Electrification                         633.8
High Efficiency Heat Pump with Electric Heat Backup                486.5
Enhanced Enclosure                                                 282.1
Basic Enclosure                                                    243.6
Mininum Efficiency Heat Pump with Existing Heat Backup             191.5
Heat Pump Water Heater                                              99.2
Mininum Efficiency Heat Pump with Electric Heat Backup              88.9
Heat Pump Clothes Dryer                                             23.6
Induction Cooking                                                   -2.0
Electric Cooking                      

### Annual Energy Savings Per Dwelling Unit

In [220]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    77.3
Basic Enclosure + High Efficiency Whole Home Electrification       76.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     74.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        73.8
High Efficiency Whole Home Electrification                         69.2
High Efficiency Heat Pump with Electric Heat Backup                54.2
Mininum Efficiency Whole Home Electrification                      41.7
Mininum Efficiency Heat Pump with Electric Heat Backup             35.1
Enhanced Enclosure                                                 31.0
Mininum Efficiency Heat Pump with Existing Heat Backup             30.2
Basic Enclosure                                                    26.8
Heat Pump Water Heater                                              8.4
Electric Cooking                                                    1.8
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    44.0
Basic Enclosure + High Efficiency Whole Home Electrification       43.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     43.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        42.7
Mininum Efficiency Whole Home Electrification                      42.3
High Efficiency Whole Home Electrification                         39.1
High Efficiency Heat Pump with Electric Heat Backup                32.9
Mininum Efficiency Heat Pump with Electric Heat Backup             21.1
Mininum Efficiency Heat Pump with Existing Heat Backup             16.8
Enhanced Enclosure                                                 16.6
Basic Enclosure                                                    15.4
Heat Pump Water Heater                                              3.2
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    43.3
Basic Enclosure + High Efficiency Whole Home Electrification       42.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     42.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        41.7
High Efficiency Whole Home Electrification                         39.6
Mininum Efficiency Whole Home Electrification                      33.1
High Efficiency Heat Pump with Electric Heat Backup                30.8
Mininum Efficiency Heat Pump with Electric Heat Backup             19.3
Enhanced Enclosure                                                 14.5
Basic Enclosure                                                    12.5
Mininum Efficiency Heat Pump with Existing Heat Backup             11.6
Heat Pump Water Heater                                              6.0
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [221]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    56.1
Basic Enclosure + High Efficiency Whole Home Electrification       54.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     52.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.1
High Efficiency Whole Home Electrification                         43.3
High Efficiency Heat Pump with Electric Heat Backup                28.9
Enhanced Enclosure                                                 23.8
Mininum Efficiency Heat Pump with Existing Heat Backup             23.8
Basic Enclosure                                                    21.0
Heat Pump Water Heater                                              8.4
Electric Cooking                                                    2.7
Mininum Efficiency Heat Pump with Electric Heat Backup              2.0
Induction Cooking                                                   1.8
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    62.9
Basic Enclosure + High Efficiency Whole Home Electrification       62.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     61.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        60.9
High Efficiency Whole Home Electrification                         56.1
Mininum Efficiency Whole Home Electrification                      51.1
High Efficiency Heat Pump with Electric Heat Backup                46.9
Mininum Efficiency Heat Pump with Electric Heat Backup             29.1
Mininum Efficiency Heat Pump with Existing Heat Backup             24.9
Enhanced Enclosure                                                 22.3
Basic Enclosure                                                    19.9
Heat Pump Water Heater                                              4.8
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.7
Basic Enclosure + High Efficiency Whole Home Electrification       53.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     51.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.3
High Efficiency Whole Home Electrification                         48.4
High Efficiency Heat Pump with Electric Heat Backup                35.9
Mininum Efficiency Whole Home Electrification                      28.6
Mininum Efficiency Heat Pump with Electric Heat Backup             18.9
Enhanced Enclosure                                                 16.1
Basic Enclosure                                                    14.0
Mininum Efficiency Heat Pump with Existing Heat Backup             13.5
Heat Pump Water Heater                                              8.7
Electric Cooking                                                    2.3
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [222]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5386.7
Basic Enclosure + High Efficiency Whole Home Electrification       5306.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5212.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5132.9
High Efficiency Whole Home Electrification                         4706.9
High Efficiency Heat Pump with Electric Heat Backup                3659.7
Mininum Efficiency Whole Home Electrification                      2330.2
Enhanced Enclosure                                                 2223.2
Mininum Efficiency Heat Pump with Existing Heat Backup             2056.2
Mininum Efficiency Heat Pump with Electric Heat Backup             2051.7
Basic Enclosure                                                    1920.2
Heat Pump Water Heater                                              584.8
Electric Cooking                                                    113.7
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3361.0
Basic Enclosure + High Efficiency Whole Home Electrification       3326.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3305.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3271.1
High Efficiency Whole Home Electrification                         2945.0
High Efficiency Heat Pump with Electric Heat Backup                2511.4
Mininum Efficiency Whole Home Electrification                      2420.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1518.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1314.6
Enhanced Enclosure                                                 1282.7
Basic Enclosure                                                    1179.2
Heat Pump Water Heater                                              215.6
Electric Cooking                                                     94.5
Heat Pump Clothes Dryer  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3127.5
Basic Enclosure + High Efficiency Whole Home Electrification       3095.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3039.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3008.2
High Efficiency Whole Home Electrification                         2807.9
High Efficiency Heat Pump with Electric Heat Backup                2192.6
Mininum Efficiency Whole Home Electrification                      1929.9
Mininum Efficiency Heat Pump with Electric Heat Backup             1223.6
Enhanced Enclosure                                                 1080.1
Basic Enclosure                                                     926.2
Mininum Efficiency Heat Pump with Existing Heat Backup              850.3
Heat Pump Water Heater                                              417.0
Electric Cooking                                                    100.0
Heat Pump Clothes Dryer  

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [223]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [224]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [225]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [226]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]

energy burden (%) before and after upgrade
segment1


C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3366644941.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3366644941.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3366644941.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\ipykernel

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.3,5.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.3,5.8
Basic Enclosure + High Efficiency Whole Home Electrification,10.3,5.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.3,6.0
High Efficiency Whole Home Electrification,10.3,7.0
High Efficiency Heat Pump with Electric Heat Backup,10.3,7.8
Enhanced Enclosure,10.3,8.1
Basic Enclosure,10.3,8.4
Mininum Efficiency Heat Pump with Existing Heat Backup,11.6,10.8


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.1,6.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.1,6.8
Basic Enclosure + High Efficiency Whole Home Electrification,15.1,7.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.1,7.2
High Efficiency Whole Home Electrification,15.1,8.2
High Efficiency Heat Pump with Electric Heat Backup,15.1,9.4
Enhanced Enclosure,15.1,11.5
Basic Enclosure,15.1,12.3
Mininum Efficiency Heat Pump with Existing Heat Backup,15.1,12.4


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.4,5.9
Basic Enclosure + High Efficiency Whole Home Electrification,10.4,6.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.4,6.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.4,6.1
High Efficiency Whole Home Electrification,10.4,6.6
High Efficiency Heat Pump with Electric Heat Backup,10.6,7.8
Enhanced Enclosure,10.6,9.1
Basic Enclosure,10.5,9.2
Mininum Efficiency Heat Pump with Existing Heat Backup,10.4,9.4


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single family detached, pre 1940, renters, 0-80% AMI 

Segment 2. Occupied, Multifamily buildings, pre 1940, renters, 0-80% AMI 

Segment 3. Occupied, All housing types, renters, 0-80% AMI


In [227]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

In [228]:
income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Multi-Family with 2 - 4 Units', 'Multi-Family with 5+ Units']
seg2_vintage = ['<1940']
seg2_tenure = ['Renter']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_tenure = ['Renter']
seg3_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &
                    community_results['build_existing_model.area_median_income'].isin(seg1_median_income) &
                     community_results['build_existing_model.tenure'].isin(seg1_tenure),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) &
                      community_results['build_existing_model.area_median_income'].isin(seg2_median_income) &
                     community_results['build_existing_model.tenure'].isin(seg2_tenure),
                     'segment'] = 'segment2'

seg3 = community_results.copy().assign(segment='segment3')
seg3 = seg3.loc[seg3['build_existing_model.tenure'].isin(seg3_tenure) &
                seg3['build_existing_model.area_median_income'].isin(seg3_median_income) &
                seg3['build_existing_model.vacancy_status'].isin(vacancy_status)]
community_results = pd.concat([community_results, seg3], axis=0)

In [229]:
metric = 'saving_cbill.total_usd'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    results = get_summary(df_seg, metric, lower_percentile=0.10, middle_percentile1=0.25, middle_percentile2=0.75, upper_percentile=0.90)
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.1'] = results.iloc[:, 3]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.25'] = results.iloc[:, 4]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.75'] = results.iloc[:, 5]
    saving_results[f'{segment}_{income}_saving_bill_total_usd_wt_p0.9'] = results.iloc[:, 6]
    display(results)

Summary for segment: segment1: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,19,463.351509,871.633522,-215.039362,625.497524,1331.758929,1817.745312
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,19,463.351509,846.540211,-212.223831,594.401410,1315.985346,1780.466666
Basic Enclosure + High Efficiency Whole Home Electrification,19,463.351509,830.735070,-307.489792,625.497524,1308.074323,1817.745312
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,19,463.351509,805.641758,-304.674261,594.401410,1283.333096,1780.466666
High Efficiency Whole Home Electrification,19,463.351509,562.834172,-643.999438,430.805420,934.010660,1619.183218
Enhanced Enclosure,19,463.351509,457.999564,66.236935,224.377026,662.658139,778.929367
High Efficiency Heat Pump with Electric Heat Backup,19,463.351509,400.217129,-680.317656,217.803434,710.456624,1198.968505
Basic Enclosure,19,463.351509,394.332683,57.070379,199.675731,584.772524,702.002854
Mininum Efficiency Heat Pump with Existing Heat Backup,18,385.037949,157.785098,-148.545882,-33.430589,226.651491,746.182441


Summary for segment: segment2: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,5,202.310028,1070.242967,540.225657,578.037942,1527.758846,2047.329660
Basic Enclosure + High Efficiency Whole Home Electrification,5,202.310028,1055.114809,540.225657,578.037942,1469.137230,2047.329660
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,5,202.310028,1051.374838,537.459713,565.015838,1489.092789,2012.052935
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,5,202.310028,1036.246679,537.459713,565.015838,1430.471174,2012.052935
High Efficiency Whole Home Electrification,5,202.310028,902.307017,343.112961,402.856627,1286.718522,1783.860948
High Efficiency Heat Pump with Electric Heat Backup,5,202.310028,777.299932,253.522748,330.474102,1099.700005,1654.132523
Enhanced Enclosure,5,202.310028,424.689454,71.383489,137.655068,716.325015,902.323274
Mininum Efficiency Heat Pump with Existing Heat Backup,5,202.310028,411.642486,-15.131826,63.186725,768.529187,1049.032945
Basic Enclosure,5,202.310028,382.683153,71.383489,138.080708,611.255989,902.323274


Summary for segment: segment3: 


,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.1,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75,saving_cbill.total_usd wt_p0.9
upgrade_name,,,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,195,6183.564070,738.135527,236.062659,380.382790,908.805701,1410.171721
Basic Enclosure + High Efficiency Whole Home Electrification,195,6183.564070,725.083772,228.571197,380.046821,890.134235,1363.454800
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,195,6183.564070,721.479844,217.203887,367.419169,875.954054,1360.323969
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,195,6183.564070,708.428089,210.396424,360.076190,855.059835,1360.323969
High Efficiency Whole Home Electrification,195,6183.564070,612.921614,178.238059,323.736192,739.838903,1328.533417
High Efficiency Heat Pump with Electric Heat Backup,194,6124.828901,457.311076,109.383511,213.143927,553.163602,1002.242854
Enhanced Enclosure,193,6140.847584,264.038453,41.396527,86.553722,334.428038,587.135774
Basic Enclosure,192,6093.859448,224.265385,36.934424,74.639199,267.458826,487.193969
Mininum Efficiency Heat Pump with Existing Heat Backup,192,6025.157098,181.558055,-34.978024,24.263288,186.887478,696.592220


## 3.1 Upgrade Cost Per Dwelling Unit 

In [230]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Heat Pump Water Heater                                              3224.1
Basic Enclosure                                                     7173.8
Enhanced Enclosure                                                 10300.2
Mininum Efficiency Heat Pump with Existing Heat Backup             21191.5
Mininum Efficiency Heat Pump with Electric Heat Backup             28283.9
Mininum Efficiency Whole Home Electrification                      29977.1
High Efficiency Heat Pump with Electric Heat Backup                34412.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        37835.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     38913.9
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Basic Enclosure                                                     2927.6
Heat Pump Water Heater                                              2964.4
Enhanced Enclosure                                                  3242.4
Mininum Efficiency Whole Home Electrification                      13575.0
Mininum Efficiency Heat Pump with Electric Heat Backup             14153.2
Mininum Efficiency Heat Pump with Existing Heat Backup             14153.2
High Efficiency Heat Pump with Electric Heat Backup                24100.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28268.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     28456.4
High Efficiency Whole Home Electrification                         30223.0
Basic Enclos

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1150.3
Electric Clothes Dryer                                              1291.8
Induction Cooking                                                   1978.0
Heat Pump Clothes Dryer                                             2599.4
Basic Enclosure                                                     2692.2
Heat Pump Water Heater                                              3035.0
Enhanced Enclosure                                                  4108.7
Mininum Efficiency Heat Pump with Existing Heat Backup             12215.2
Mininum Efficiency Heat Pump with Electric Heat Backup             12926.3
Mininum Efficiency Whole Home Electrification                      13687.5
High Efficiency Heat Pump with Electric Heat Backup                25345.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        28866.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     29630.4
High Efficie

 ## 3.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [231]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]

segment1


upgrade_name
Basic Enclosure                                                    0.05497
Enhanced Enclosure                                                 0.04447
Heat Pump Water Heater                                             0.02664
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02175
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02004
Basic Enclosure + High Efficiency Whole Home Electrification       0.01959
High Efficiency Whole Home Electrification                         0.01333
Heat Pump Clothes Dryer                                            0.01187
High Efficiency Heat Pump with Electric Heat Backup                0.01163
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00745
Induction Cooking                                                 -0.00487
Mininum Efficiency Heat Pump with Electric Heat Backup            -0.01190
Mininum Effi

segment2


upgrade_name
Enhanced Enclosure                                                 0.13098
Basic Enclosure                                                    0.13071
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03695
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03666
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03385
Basic Enclosure + High Efficiency Whole Home Electrification       0.03357
High Efficiency Heat Pump with Electric Heat Backup                0.03225
High Efficiency Whole Home Electrification                         0.02985
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02908
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02596
Heat Pump Water Heater                                             0.01767
Heat Pump Clothes Dryer                                            0.01159
Induction Cooking                                                  0.00014
Mininum Effi

segment3


upgrade_name
Basic Enclosure                                                    0.08330
Enhanced Enclosure                                                 0.06426
Heat Pump Water Heater                                             0.03151
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02454
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02435
Basic Enclosure + High Efficiency Whole Home Electrification       0.02228
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02216
High Efficiency Whole Home Electrification                         0.01927
High Efficiency Heat Pump with Electric Heat Backup                0.01804
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01486
Heat Pump Clothes Dryer                                            0.00950
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00590
Induction Cooking                                                 -0.00108
Mininum Effi

 ### Annual Energy Savings Per Dollar Invested

In [232]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]

segment1


upgrade_name
Basic Enclosure                                                    0.00387
Enhanced Enclosure                                                 0.00323
Heat Pump Water Heater                                             0.00264
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00207
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00204
Basic Enclosure + High Efficiency Whole Home Electrification       0.00190
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00188
High Efficiency Whole Home Electrification                         0.00173
Mininum Efficiency Whole Home Electrification                      0.00171
High Efficiency Heat Pump with Electric Heat Backup                0.00166
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00147
Electric Cooking                                                   0.00146
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00127
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.00525
Enhanced Enclosure                                                 0.00512
Mininum Efficiency Whole Home Electrification                      0.00311
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00152
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00151
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00149
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00139
Basic Enclosure + High Efficiency Whole Home Electrification       0.00139
High Efficiency Heat Pump with Electric Heat Backup                0.00137
Electric Cooking                                                   0.00132
High Efficiency Whole Home Electrification                         0.00129
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00119
Heat Pump Water Heater                                             0.00108
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.00436
Enhanced Enclosure                                                 0.00335
Mininum Efficiency Whole Home Electrification                      0.00247
Heat Pump Water Heater                                             0.00191
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00142
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00141
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00139
Electric Cooking                                                   0.00138
Basic Enclosure + High Efficiency Whole Home Electrification       0.00129
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00127
High Efficiency Whole Home Electrification                         0.00122
High Efficiency Heat Pump with Electric Heat Backup                0.00116
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00094
Electric Clo

### Annual Emission Savings Per Dollar Invested

In [233]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]

segment1


upgrade_name
Basic Enclosure                                                    0.27313
Enhanced Enclosure                                                 0.22847
Heat Pump Water Heater                                             0.17944
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.14114
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13989
Basic Enclosure + High Efficiency Whole Home Electrification       0.12960
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12877
High Efficiency Whole Home Electrification                         0.11481
High Efficiency Heat Pump with Electric Heat Backup                0.10931
Electric Cooking                                                   0.09719
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09671
Mininum Efficiency Whole Home Electrification                      0.09530
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06964
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.40280
Enhanced Enclosure                                                 0.39559
Mininum Efficiency Whole Home Electrification                      0.17830
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.11617
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.11572
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10732
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.10631
Basic Enclosure + High Efficiency Whole Home Electrification       0.10585
High Efficiency Heat Pump with Electric Heat Backup                0.10420
High Efficiency Whole Home Electrification                         0.09744
Mininum Efficiency Heat Pump with Existing Heat Backup             0.09289
Electric Cooking                                                   0.08219
Heat Pump Water Heater                                             0.07273
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.32305
Enhanced Enclosure                                                 0.24980
Mininum Efficiency Whole Home Electrification                      0.14448
Heat Pump Water Heater                                             0.13310
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.10209
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.10057
Basic Enclosure + High Efficiency Whole Home Electrification       0.09314
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.09199
Mininum Efficiency Heat Pump with Electric Heat Backup             0.09010
Electric Cooking                                                   0.08857
High Efficiency Whole Home Electrification                         0.08660
High Efficiency Heat Pump with Electric Heat Backup                0.08255
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06846
Heat Pump Cl

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [234]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    403872.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     392245.7
Basic Enclosure + High Efficiency Whole Home Electrification       384922.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        373295.3
High Efficiency Whole Home Electrification                         260790.1
Enhanced Enclosure                                                 212214.8
High Efficiency Heat Pump with Electric Heat Backup                185441.2
Basic Enclosure                                                    182714.6
Mininum Efficiency Heat Pump with Existing Heat Backup              60753.3
Heat Pump Water Heater                                              39801.4
Heat Pump Clothes Dryer                                             14298.1
Electric Clothes Dryer                                              -3608.8
Induction Cooking                                                   -4462.8

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    216520.9
Basic Enclosure + High Efficiency Whole Home Electrification       213460.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     212703.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        209643.1
High Efficiency Whole Home Electrification                         182545.8
High Efficiency Heat Pump with Electric Heat Backup                157255.6
Enhanced Enclosure                                                  85918.9
Mininum Efficiency Heat Pump with Existing Heat Backup              83279.4
Basic Enclosure                                                     77420.6
Mininum Efficiency Heat Pump with Electric Heat Backup              74342.6
Heat Pump Water Heater                                              10594.5
Heat Pump Clothes Dryer                                              3440.2
Induction Cooking                                                      46.6

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4564308.3
Basic Enclosure + High Efficiency Whole Home Electrification       4483602.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4461316.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4380610.5
High Efficiency Whole Home Electrification                         3790040.1
High Efficiency Heat Pump with Electric Heat Backup                2800952.1
Enhanced Enclosure                                                 1621419.9
Basic Enclosure                                                    1366641.7
Mininum Efficiency Heat Pump with Existing Heat Backup             1093915.8
Heat Pump Water Heater                                              589980.4
Mininum Efficiency Heat Pump with Electric Heat Backup              467187.2
Heat Pump Clothes Dryer                                             100917.4
Induction Cooking                                              

### Annual Total Energy Savings

In [235]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    37882.4
Basic Enclosure + High Efficiency Whole Home Electrification       37334.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     36827.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        36279.6
High Efficiency Whole Home Electrification                         33745.2
High Efficiency Heat Pump with Electric Heat Backup                26486.3
Mininum Efficiency Whole Home Electrification                      20266.5
Mininum Efficiency Heat Pump with Electric Heat Backup             16617.9
Enhanced Enclosure                                                 15436.7
Basic Enclosure                                                    12877.8
Mininum Efficiency Heat Pump with Existing Heat Backup             12017.2
Heat Pump Water Heater                                              3940.4
Heat Pump Clothes Dryer                                              565.3
Induction Co

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    8897.3
Basic Enclosure + High Efficiency Whole Home Electrification       8808.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     8734.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        8645.7
High Efficiency Whole Home Electrification                         7914.9
High Efficiency Heat Pump with Electric Heat Backup                6662.3
Mininum Efficiency Heat Pump with Electric Heat Backup             4265.8
Mininum Efficiency Heat Pump with Existing Heat Backup             3402.7
Enhanced Enclosure                                                 3355.5
Basic Enclosure                                                    3109.8
Mininum Efficiency Whole Home Electrification                      2481.8
Heat Pump Water Heater                                              647.1
Heat Pump Clothes Dryer                                              99.5
Induction Cooking        

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    262441.1
Basic Enclosure + High Efficiency Whole Home Electrification       260107.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     254855.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        252521.6
High Efficiency Whole Home Electrification                         240053.7
High Efficiency Heat Pump with Electric Heat Backup                179877.8
Mininum Efficiency Whole Home Electrification                      113583.6
Mininum Efficiency Heat Pump with Electric Heat Backup             112411.0
Enhanced Enclosure                                                  84586.8
Basic Enclosure                                                     71465.3
Mininum Efficiency Heat Pump with Existing Heat Backup              69144.9
Heat Pump Water Heater                                              35854.1
Heat Pump Clothes Dryer                                              3979.3

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\922227291.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\922227291.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\922227291.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

### Annual Total Emission Savings

In [236]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2594885.8
Basic Enclosure + High Efficiency Whole Home Electrification       2547002.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2522257.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2474374.1
High Efficiency Whole Home Electrification                         2245652.8
High Efficiency Heat Pump with Electric Heat Backup                1742921.6
Mininum Efficiency Whole Home Electrification                      1127989.8
Enhanced Enclosure                                                 1090398.0
Mininum Efficiency Heat Pump with Electric Heat Backup              912713.7
Basic Enclosure                                                     907891.0
Mininum Efficiency Heat Pump with Existing Heat Backup              789110.5
Heat Pump Water Heater                                              268062.5
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    679971.6
Basic Enclosure + High Efficiency Whole Home Electrification       672969.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     668782.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        661780.3
High Efficiency Whole Home Electrification                         595793.9
High Efficiency Heat Pump with Electric Heat Backup                508076.0
Mininum Efficiency Heat Pump with Electric Heat Backup             307295.4
Mininum Efficiency Heat Pump with Existing Heat Backup             265965.1
Enhanced Enclosure                                                 259493.4
Basic Enclosure                                                    238573.1
Mininum Efficiency Whole Home Electrification                      142159.9
Heat Pump Water Heater                                              43615.0
Heat Pump Clothes Dryer                                              7027.1

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    18945054.2
Basic Enclosure + High Efficiency Whole Home Electrification       18741842.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     18426588.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        18223376.7
High Efficiency Whole Home Electrification                         17033959.2
High Efficiency Heat Pump with Electric Heat Backup                12815755.1
Mininum Efficiency Heat Pump with Electric Heat Backup              7133462.3
Mininum Efficiency Whole Home Electrification                       6641233.1
Enhanced Enclosure                                                  6302663.6
Basic Enclosure                                                     5299865.3
Mininum Efficiency Heat Pump with Existing Heat Backup              5038323.8
Heat Pump Water Heater                                              2492288.0
Heat Pump Clothes Dryer                            

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4045702603.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4045702603.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4045702603.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [237]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    871.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     846.5
Basic Enclosure + High Efficiency Whole Home Electrification       830.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        805.6
High Efficiency Whole Home Electrification                         562.8
Enhanced Enclosure                                                 458.0
High Efficiency Heat Pump with Electric Heat Backup                400.2
Basic Enclosure                                                    394.3
Mininum Efficiency Heat Pump with Existing Heat Backup             157.8
Heat Pump Water Heater                                              85.9
Heat Pump Clothes Dryer                                             30.9
Induction Cooking                                                   -9.6
Electric Cooking                                                   -30.0
Electric Clothes Dryer                

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1070.2
Basic Enclosure + High Efficiency Whole Home Electrification       1055.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1051.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1036.2
High Efficiency Whole Home Electrification                          902.3
High Efficiency Heat Pump with Electric Heat Backup                 777.3
Enhanced Enclosure                                                  424.7
Mininum Efficiency Heat Pump with Existing Heat Backup              411.6
Basic Enclosure                                                     382.7
Mininum Efficiency Heat Pump with Electric Heat Backup              367.5
Heat Pump Water Heater                                               52.4
Heat Pump Clothes Dryer                                              30.1
Induction Cooking                                                     0.3
Electric Cooking         

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    738.1
Basic Enclosure + High Efficiency Whole Home Electrification       725.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     721.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        708.4
High Efficiency Whole Home Electrification                         612.9
High Efficiency Heat Pump with Electric Heat Backup                457.3
Enhanced Enclosure                                                 264.0
Basic Enclosure                                                    224.3
Mininum Efficiency Heat Pump with Existing Heat Backup             181.6
Heat Pump Water Heater                                              95.6
Mininum Efficiency Heat Pump with Electric Heat Backup              76.3
Heat Pump Clothes Dryer                                             24.7
Induction Cooking                                                   -2.1
Electric Cooking                      

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\2125560417.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\2125560417.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\2125560417.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

### Annual Energy Savings Per Dwelling Unit

In [238]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    81.8
Basic Enclosure + High Efficiency Whole Home Electrification       80.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     79.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        78.3
High Efficiency Whole Home Electrification                         72.8
High Efficiency Heat Pump with Electric Heat Backup                57.2
Mininum Efficiency Whole Home Electrification                      51.3
Mininum Efficiency Heat Pump with Electric Heat Backup             35.9
Enhanced Enclosure                                                 33.3
Mininum Efficiency Heat Pump with Existing Heat Backup             31.2
Basic Enclosure                                                    27.8
Heat Pump Water Heater                                              8.5
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    44.0
Basic Enclosure + High Efficiency Whole Home Electrification       43.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     43.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        42.7
Mininum Efficiency Whole Home Electrification                      42.3
High Efficiency Whole Home Electrification                         39.1
High Efficiency Heat Pump with Electric Heat Backup                32.9
Mininum Efficiency Heat Pump with Electric Heat Backup             21.1
Mininum Efficiency Heat Pump with Existing Heat Backup             16.8
Enhanced Enclosure                                                 16.6
Basic Enclosure                                                    15.4
Heat Pump Water Heater                                              3.2
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    42.4
Basic Enclosure + High Efficiency Whole Home Electrification       42.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     41.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        40.8
High Efficiency Whole Home Electrification                         38.8
Mininum Efficiency Whole Home Electrification                      33.8
High Efficiency Heat Pump with Electric Heat Backup                29.4
Mininum Efficiency Heat Pump with Electric Heat Backup             18.4
Enhanced Enclosure                                                 13.8
Basic Enclosure                                                    11.7
Mininum Efficiency Heat Pump with Existing Heat Backup             11.5
Heat Pump Water Heater                                              5.8
Electric Cooking                                                    1.6
Heat Pump Clothes Dryer                            

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3547008041.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3547008041.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3547008041.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

### Annual Percentage Energy Savings Per Dwelling Unit

In [239]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    52.7
Basic Enclosure + High Efficiency Whole Home Electrification       50.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     49.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        47.3
High Efficiency Whole Home Electrification                         37.5
Enhanced Enclosure                                                 23.7
High Efficiency Heat Pump with Electric Heat Backup                23.0
Mininum Efficiency Heat Pump with Existing Heat Backup             21.0
Basic Enclosure                                                    20.1
Heat Pump Water Heater                                              8.2
Electric Cooking                                                    2.9
Induction Cooking                                                   1.7
Heat Pump Clothes Dryer                                             1.3
Electric Clothes Dryer                             

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    62.9
Basic Enclosure + High Efficiency Whole Home Electrification       62.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     61.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        60.9
High Efficiency Whole Home Electrification                         56.1
Mininum Efficiency Whole Home Electrification                      51.1
High Efficiency Heat Pump with Electric Heat Backup                46.9
Mininum Efficiency Heat Pump with Electric Heat Backup             29.1
Mininum Efficiency Heat Pump with Existing Heat Backup             24.9
Enhanced Enclosure                                                 22.3
Basic Enclosure                                                    19.9
Heat Pump Water Heater                                              4.8
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    53.9
Basic Enclosure + High Efficiency Whole Home Electrification       53.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     52.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        51.6
High Efficiency Whole Home Electrification                         48.7
High Efficiency Heat Pump with Electric Heat Backup                35.0
Mininum Efficiency Whole Home Electrification                      28.3
Mininum Efficiency Heat Pump with Electric Heat Backup             18.0
Enhanced Enclosure                                                 15.6
Mininum Efficiency Heat Pump with Existing Heat Backup             13.9
Basic Enclosure                                                    13.4
Heat Pump Water Heater                                              8.6
Electric Cooking                                                    2.3
Heat Pump Clothes Dryer                            

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\2997227555.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\2997227555.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\2997227555.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

### Annual Emission Savings Per Dwelling Unit

In [240]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5600.3
Basic Enclosure + High Efficiency Whole Home Electrification       5496.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5443.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5340.2
High Efficiency Whole Home Electrification                         4846.5
High Efficiency Heat Pump with Electric Heat Backup                3761.6
Mininum Efficiency Whole Home Electrification                      2856.9
Enhanced Enclosure                                                 2353.3
Mininum Efficiency Heat Pump with Existing Heat Backup             2049.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1969.8
Basic Enclosure                                                    1959.4
Heat Pump Water Heater                                              578.5
Electric Cooking                                                    111.8
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3361.0
Basic Enclosure + High Efficiency Whole Home Electrification       3326.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3305.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3271.1
High Efficiency Whole Home Electrification                         2945.0
High Efficiency Heat Pump with Electric Heat Backup                2511.4
Mininum Efficiency Whole Home Electrification                      2420.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1518.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1314.6
Enhanced Enclosure                                                 1282.7
Basic Enclosure                                                    1179.2
Heat Pump Water Heater                                              215.6
Electric Cooking                                                     94.5
Heat Pump Clothes Dryer  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3063.8
Basic Enclosure + High Efficiency Whole Home Electrification       3030.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2979.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2947.1
High Efficiency Whole Home Electrification                         2754.7
High Efficiency Heat Pump with Electric Heat Backup                2092.4
Mininum Efficiency Whole Home Electrification                      1977.6
Mininum Efficiency Heat Pump with Electric Heat Backup             1164.7
Enhanced Enclosure                                                 1026.4
Basic Enclosure                                                     869.7
Mininum Efficiency Heat Pump with Existing Heat Backup              836.2
Heat Pump Water Heater                                              404.0
Electric Cooking                                                    101.9
Heat Pump Clothes Dryer  

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3299577571.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3299577571.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3299577571.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [241]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

### Segment 2

In [242]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

### Segment 3

In [243]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4148934776.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4148934776.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4148934776.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [244]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]

energy burden (%) before and after upgrade
segment1


C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3366644941.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3366644941.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\3366644941.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ylou2\AppData\Local\Temp\ipykernel

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12.2,7.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12.2,7.2
Basic Enclosure + High Efficiency Whole Home Electrification,12.2,7.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12.2,7.4
High Efficiency Whole Home Electrification,12.2,8.6
High Efficiency Heat Pump with Electric Heat Backup,12.2,9.5
Enhanced Enclosure,12.2,9.6
Basic Enclosure,12.2,10.0
Mininum Efficiency Heat Pump with Existing Heat Backup,14.4,13.8


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.1,6.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.1,6.8
Basic Enclosure + High Efficiency Whole Home Electrification,15.1,7.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.1,7.2
High Efficiency Whole Home Electrification,15.1,8.2
High Efficiency Heat Pump with Electric Heat Backup,15.1,9.4
Enhanced Enclosure,15.1,11.5
Basic Enclosure,15.1,12.3
Mininum Efficiency Heat Pump with Existing Heat Backup,15.1,12.4


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12.1,6.9
Basic Enclosure + High Efficiency Whole Home Electrification,12.1,7.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12.1,7.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12.1,7.1
High Efficiency Whole Home Electrification,12.1,7.7
High Efficiency Heat Pump with Electric Heat Backup,12.1,8.9
Enhanced Enclosure,12.1,10.5
Basic Enclosure,11.9,10.5
Mininum Efficiency Heat Pump with Existing Heat Backup,12.1,10.9


C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4220242559.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4220242559.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\ylou2\AppData\Local\Temp\ipykernel_12568\4220242559.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

In [245]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')

### Save html file

In [246]:
! jupyter nbconvert --output-dir='./results/jackson_county/' --to=html --no-input saving_results_reports_for_jackson_county.ipynb

C:\ProgramData\Anaconda3\lib\site-packages\traitlets\traitlets.py:2202: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './results/jackson_county/' instead of "'./results/jackson_county/'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook saving_results_reports_for_jackson_county.ipynb to html
[NbConvertApp] Writing 948521 bytes to results\jackson_county\saving_results_reports_for_jackson_county.html
